In [ ]:
#  RunnableLambda

In [2]:
from langchain_core.runnables import RunnableLambda

double = RunnableLambda(lambda x: x * 2)
print(double.invoke(4))  # Output: 8



8


# RunnableSequence (|)

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("Translate this to French: {text}")
llm = ChatOpenAI(model="gpt-4o-mini")

chain = prompt | llm
print(chain.invoke({"text": "Hello"}))



content='Hello in French is "Bonjour."' response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 13, 'total_tokens': 20, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'finish_reason': 'stop', 'logprobs': None} id='run-4779cecb-ad33-4117-aede-dcb72e741b66-0' usage_metadata={'input_tokens': 13, 'output_tokens': 7, 'total_tokens': 20}


# RunnableParallel

In [4]:
from langchain_core.runnables import RunnableParallel

par = RunnableParallel(
    english=lambda x: x,
    french=lambda x: f"Bonjour {x}"
)

print(par.invoke("World"))
# Output: {'english': 'World', 'french': 'Bonjour World'}



{'english': 'World', 'french': 'Bonjour World'}


# RunnableMap

In [ ]:
from langchain_core.runnables import RunnableMap

mapper = RunnableMap({
    "upper": lambda x: x.upper(),
    "lower": lambda x: x.lower()
})

print(mapper.invoke("Hello"))
# Output: {'upper': 'HELLO', 'lower': 'hello'}



RunnableBranch

In [11]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [12]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo-0125")

RunnablePassthrough

It does not do anything to the input data.
Let's see it in a very simple example: a chain with just RunnablePassthrough() will output the original input without any modification.

In [13]:
from langchain_core.runnables import RunnablePassthrough

chain = RunnablePassthrough()

In [14]:
chain.invoke("selvan")


'selvan'

### RunnableLambda
To use a custom function inside a LCEL chain we need to wrap it up with RunnableLambda.
Let's define a very simple function to create Russian lastnames:

In [15]:
def russian_lastname(name: str) -> str:
    return f"{name}ovich"

In [16]:
from langchain_core.runnables import RunnableLambda

chain = RunnablePassthrough() | RunnableLambda(russian_lastname)

In [17]:
chain.invoke("selvan ")

'selvan ovich'

### RunnableParallel

We will use RunnableParallel() for running tasks in parallel.
This is probably the most important and most useful Runnable from LangChain.
In the following chain, RunnableParallel is going to run these two tasks in parallel:
operation_a will use RunnablePassthrough.
operation_b will use RunnableLambda with the russian_lastname function.


In [18]:
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "operation_b": RunnableLambda(russian_lastname)
    }
)

In [19]:
chain.invoke("tamil")


{'operation_a': 'tamil', 'operation_b': 'tamilovich'}

In [20]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [21]:
prompt = ChatPromptTemplate.from_template("tell me a curious fact about {soccer_player}")

output_parser = StrOutputParser()

In [22]:
def russian_lastname_from_dictionary(person):
    return person["name"] + "ovich"

In [23]:
chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "soccer_player": RunnableLambda(russian_lastname_from_dictionary),
        "operation_c": RunnablePassthrough(),
    }
) | prompt | model | output_parser

In [25]:
chain.invoke({
    "name1": "Jordam",
    "name": "putin "
})

'One curious fact about Vladimir Putin is that he has a black belt in judo and has practiced the martial art since he was a teenager. Putin has even written a book about his experiences with judo, highlighting its importance in his personal and political life.'

### Let's see a more advanced use of RunnableParallel


In [26]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    ["dswithbappy focuses on providing content on Data Science, AI, ML, DL, CV, NLP, Python programming, etc. in English."], embedding=OpenAIEmbeddings()
)

retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(model="gpt-3.5-turbo")

retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | model
    | StrOutputParser()
)

retrieval_chain.invoke("What is dswithbappy?")

'dswithbappy is a platform that focuses on providing content on Data Science, AI, ML, DL, CV, NLP, Python programming, etc. in English.'

In [27]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

model = ChatOpenAI(model="gpt-3.5-turbo")

vectorstore = FAISS.from_texts(
    ["dswithbappy focuses on providing content on Data Science, AI, ML, DL, CV, NLP, Python programming, etc. in English."], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""

prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke({"question": "What is dswithbappy?", "language": "Pirate English"})

"Arrr, me hearties! dswithbappy be a scallywag focusin' on providin' content on Data Science, AI, ML, DL, CV, NLP, Python programmin', etc. in English. Aye, ye can learn a lot from that scurvy dog!"